In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

from huggingface_hub import login
import os
os.environ["HF_TOKEN"] = "HF_TOKEN"

C:\Users\hamdy\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model = "google/gemma-3-1b-it"
peft_model_path = "gemma_medical_qa/checkpoint-2188"  # best checkpoint

In [3]:
# Load base + adapter
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype="auto", device_map="auto")
model = PeftModel.from_pretrained(model, peft_model_path)
model = model.merge_and_unload()  # merges LoRA weights into base

`torch_dtype` is deprecated! Use `dtype` instead!


In [4]:
# Save full merged model
merged_path = "gemma_medical_merged"
model.save_pretrained(merged_path)
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.save_pretrained(merged_path)

print(f"✅ Model merged and saved at: {merged_path}")


✅ Model merged and saved at: gemma_medical_merged


In [3]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else "cpu"
print(f"🔥 Using device: {device}")

pipe = pipeline(
    "text-generation",
    model="gemma_medical_merged",
    tokenizer="gemma_medical_merged",
    device=device,             # explicitly set GPU
    model_kwargs={"torch_dtype": torch.bfloat16 if torch.cuda.is_available() else torch.float32}
)

prompt = """Question: What are the symptoms of vitamin D deficiency?
Answer:"""

output = pipe(prompt, max_new_tokens=150, temperature=0.7, top_p=0.9)
print(output[0]["generated_text"])


🔥 Using device: 0


Device set to use cuda:0


Question: What are the symptoms of vitamin D deficiency?
Answer: Vitamin D deficiency can cause a wide range of symptoms, including:
* Bone pain
* Muscle weakness
* Impaired growth
* Impaired immunity
* High blood pressure
* Increased risk of heart disease
* Increased risk of stroke

It is important to consult a healthcare provider if you are concerned about your vitamin D levels or if you have symptoms of vitamin D deficiency.

Disclaimer: This information is intended for general knowledge and informational purposes only and does not constitute medical advice. It is essential to consult with a qualified healthcare provider for any health concerns or before making any decisions related to your health or treatment.
